# EXTRACCIÓN DE VUELOS

## En este Notebook se va a desarrollar el proceso de extracción de todos los vuelos salientes del *Aeropuerto Adolfo Suárez Madrid-Barajas* en el periodo comprendido entre el 01-11-2022 y el 31-10-2023, es decir, 1 año completo.

## Para ello se va a utilizar la técnica de extracción de datos "webscrapping" sobre la web [flightera](https://www.flightera.net/en/airport/Madrid/LEMD/departure) que mantiene registros de vuelos desde al menos el año 2017.

### Las librerías que se van a emplear principalmente para el proceso son:
- **selenium** 
- **pandas** 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
import pandas as pd
import re

In [2]:
import threading
import time
import numpy as np
import pylab as plt   
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta


In [3]:
from joblib import Parallel, delayed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select   # seleccion de un dropdown
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

from joblib._parallel_backends import LokyBackend
import asyncio

from tqdm.notebook import tqdm


In [ ]:
# # Definir las fechas de inicio y fin
# start_date = datetime(2017, 11, 1)
# end_date = datetime(2022, 11, 1)

# # Función para generar enlaces
# def generate_links(start_date, end_date):
#     links = []
    
#     # Iterar sobre las fechas en el rango
#     current_date = start_date
#     while current_date <= end_date:
#         # Formatear la fecha en el formato deseado
#         formatted_date = current_date.strftime("%Y-%m-%d")

#         # Construir el enlace y agregarlo a la lista
#         link = f"https://www.flightera.net/en/airport/Madrid/LEMD/departure/{formatted_date}%2000_00"
#         links.append(link)

#         # Avanzar a la siguiente fecha
#         current_date += timedelta(days=1)

#     return links

# # Generar la lista de enlaces
# links = generate_links(start_date, end_date)


In [4]:
links = [
'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2020-01-01%2000_00',
'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2021-01-01%2000_00',
'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2022-01-01%2000_00']

In [5]:
links

['https://www.flightera.net/en/airport/Madrid/LEMD/departure/2020-01-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2021-01-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2022-01-01%2000_00']

In [ ]:
match = re.search(r'\d{4}', links[0])
match = match.group()
match

## Para evitar problemas en el proceso de extracción se van a activar las opciones de **adblock** (previa descarga del driver correspondiente) y de las **cockies**.

In [ ]:
# Options
opciones=Options()

opciones.add_extension('drivers/adblock.crx')       # adblocker
opciones.add_argument('cookies=cookies')    # man



In [ ]:
flights =[]

In [6]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    #Columnas tras la extracción
    #columns = ["bad_date","1", "status","cod_flight","airliner", "destiny", "depart","delay_d","8","arrival","delay_a","duration","12"]  # Añade las columnas
    #Fecha de la URL
     
    match = re.search(r'\d{4}', url)
    match = match.group()
    try:
        #Opciones
        opciones=Options()
        opciones.add_extension('drivers/adblock.crx')       # adblocker
        opciones.add_argument('cookies=cookies')    # man
#         opciones.add_argument(f'--load-extension={drivers/adblock.crx}')
        #Inicializo el drive
        driver = webdriver.Chrome(options=opciones)
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        time.sleep(35)
        
        #Aceptar cookies
        
        aceptar = driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
        aceptar.click()
        
        # Extraer la información la primera vez
        table += [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]
        time.sleep(2)
        
        # Clickar en el botón Later Flights para abrir una nueva URL
        aceptar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div[6]/div[2]/a')))
        #aceptar = driver.find_element(By.XPATH, '/html/body/div[3]/main/div[6]/div[2]/a')
        aceptar.click()
        time.sleep(2)
        
        # Clickar en boton popup 
        aceptar2 = driver.find_element(By.XPATH, '/html/body/div[9]/div/div/div/div/div/div[3]/span[1]/button')
        aceptar2.click()
        time.sleep(2)
               
        aceptar = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[6]/div[2]/a')
        aceptar.click()
        time.sleep(2)
        # Bucle while hasta que la fecha de cada link desaparezca de la URL

        while match in driver.current_url:
            try:
                #Extraer datos
                table += [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]
                # Clickar en el botón Later Flights para abrir una nueva URL
                aceptar = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[6]/div[2]/a')
                aceptar.click()
        
            except:
                # Clickar en boton popup si aparece
                aceptar2 = driver.find_element(By.XPATH,'/html/body/div[9]/div/div/div/div/div/div[3]/span[1]/button')
                aceptar2.click()       

        return table  # Devuelve un DataFrame con los datos de ese día 
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        
        # En caso de error, simplemente se devolverá la lista vacía 'table'
        return table 
    finally:
        driver.quit()

In [7]:
%%time
paralelo = Parallel(n_jobs=3, verbose=True)

flights = paralelo(delayed(extraer)(url) for url in tqdm(links))

  0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


CPU times: total: 438 ms
Wall time: 3h 55min 32s


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 235.5min finished


In [32]:
flights = [elemento for sublist in flights for elemento in sublist]
len(flights)

630

In [33]:
pd.set_option('display.max_rows', None)
columns =["bad_date","1", "status","cod_flight","airliner", "destiny", "depart","delay_d","8","arrival","delay_a","duration",'12']

In [15]:
len(flights[flights.bad_date.str.endswith("2022")])

144575

In [34]:
flights = pd.DataFrame(flights, columns=columns)



In [35]:
flights

,bad_date,1,status,cod_flight,airliner,destiny,depart,delay_d,8,arrival,delay_a,duration,12
0,"Mon, 31. Oct 2022",,Landed,AV47 AVA047,Avianca,AV/AVA Bogota (BOG / SKBO) 00:25 CET,30. Oct 23:25 UTC 01:20 CET,55min late,,04:57 UTC-05,7min late,None,None
1,"Mon, 31. Oct 2022",,Landed,LA2485 LPE2485,LATAM Airlines Chile,LA/LAN Lima (LIM / SPJC) 00:40 CET,30. Oct 23:40 UTC 00:52 CET,12min late,,05:54 UTC-05,30min early,None,None
2,"Mon, 31. Oct 2022",,Landed,UX15 AEA015,Air Europa,UX/AEA San Pedro Sula (SAP / MHLM) 01:35 CET,00:35 UTC 02:02 CET,27min late,,05:28 CST,46min early,None,None
3,"Mon, 31. Oct 2022",,Landed,UX39 AEA039,Air Europa,UX/AEA Quito (UIO / SEQM) 01:45 CET,00:45 UTC 02:50 CET,1h 5min late,,07:17 UTC-05,52min late,None,None
4,"Mon, 31. Oct 2022",,Landed,KL1698 KLM22A,KLM Royal Dutch Airlines,KL/KLM Amsterdam (AMS / EHAM) 06:00 CET,05:00 UTC 06:13 CET,13min late,,08:12 CET,27min early,None,None
5,,None,None,None,None,None,None,None,None,None,None,None,None
6,"Mon, 31. Oct 2022",,Landed,LH1121 DLH21P,Lufthansa,LH/DLH Frankfurt (FRA / EDDF) 06:15 CET,05:15 UTC 07:00 CET,45min late,,08:58 CET,3min late,None,None
7,"Mon, 31. Oct 2022",,Landed,IB432 IBE04RW,Iberia,IB/IBE Bilbao (BIO / LEBB) 06:20 CET,05:20 UTC 06:32 CET,12min late,,07:09 CET,15min early,None,None
8,"Mon, 31. Oct 2022",,Landed,SN3732 BEL2FR,Brussels Airlines,SN/BEL Brussels (BRU / EBBR) 06:25 CET,05:25 UTC 06:24 CET,on time,,08:33 CET,17min early,None,None
9,"Mon, 31. Oct 2022",,Landed,FR6703 RYR80SJ,Ryanair,FR/RYR Luqa (MLA / LMML) 06:25 CET,05:25 UTC 06:39 CET,14min late,,08:56 CET,13min early,None,None


## Guardamos los datos en crudo en un archivo *.csv*

In [36]:
flights.to_csv("../data/flights/raw_flights3.csv", index=False)